In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from lightgbm import LGBMClassifier

import warnings
warnings.simplefilter('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [5]:
# Load dataset

train = pd.read_csv('../data/titanic_data/train.csv')
test = pd.read_csv('../data/titanic_data/test.csv')

print('Training Set')
train.shape
train.head()

print('\n\n')
print('Testing Set')
test.shape
test.head()

Training Set


(891, 12)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S





Testing Set


(418, 11)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
test['Survived'] = np.nan
full = pd.concat([train, test], sort=False)

full.shape
full.head()

(1309, 12)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
# Feature engineering (same as before)
full['Title'] = full['Name'].str.extract(r',\\s*([^\\.]*)\\s*\\.', expand=False)
full['Title'] = full['Title'].replace(
    ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],
    'Rare'
)
full['Title'] = full['Title'].map({
    'Mr': 1, 'Miss': 2, 'Mrs': 3, 'Master': 4, 'Rare': 5
}).fillna(0).astype(int)

full['FamilySize'] = full['SibSp'] + full['Parch'] + 1
full['IsAlone'] = (full['FamilySize'] == 1).astype(int)

In [13]:
# full['Age'] = full.groupby('Title')['Age'].apply(lambda x: x.fillna(x.median()))

full['Age'] = full['Age'].fillna(full.groupby('Title')['Age'].transform('median'))

In [14]:
full['Embarked'] = full['Embarked'].fillna(full['Embarked'].mode()[0])
full['Fare'] = full['Fare'].fillna(full['Fare'].median())

In [15]:
full['Sex'] = full['Sex'].map({'male': 0, 'female': 1}).astype(int)
full['Embarked'] = full['Embarked'].map({'S':0, 'C':1, 'Q':2}).astype(int)

features = [
    'Pclass', 'Sex', 'Age', 'Fare', 'Embarked',
    'Title', 'FamilySize', 'IsAlone'
]

train_df = full[full['Survived'].notnull()]
test_df = full[full['Survived'].isnull()]

train_df.shape
train_df.head()

print('\n\n')

test_df.shape
test_df.head()

(891, 15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,0,2,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,0,2,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,0,2,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,0,1,1


(418, 15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,IsAlone
0,892,NaN,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,2,0,1,1
1,893,NaN,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,0,0,2,0
2,894,NaN,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,2,0,1,1
3,895,NaN,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,0,0,1,1
4,896,NaN,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,0,0,3,0


In [16]:
X = train_df[features]
y = train_df['Survived'].astype(int)

X_test = test_df[features]

In [17]:
# Train/test split for validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2
    , random_state=27
    , stratify=y
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((712, 8), (179, 8), (712,), (179,))

In [22]:
# Instantiate individual models

rf_model = RandomForestClassifier(
    n_estimators=100
    , max_depth=6
    , random_state=27
)

xgb_model = xgb.XGBClassifier(
    n_estimators=200
    , max_depth=4
    , learning_rate=0.05
#     , subsample=0.8
#     , colsample_bytree=0.8
    , use_label_encoder=False
    , eval_metric='logloss'
    , random_state=27
)

lgbm_model = LGBMClassifier(
    n_estimators=200
    , max_depth=6
    , learning_rate=0.05
#     , subsample=0.8
#     , colsample_bytree=0.8
    , random_state=27
)

In [23]:
# Fit models individually

rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,6
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003354 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 199
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,6
,learning_rate,0.05
,n_estimators,200
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [26]:
# Make predictions
rf_preds = rf_model.predict(X_val)
xgb_preds = xgb_model.predict(X_val)
lgbm_preds = lgbm_model.predict(X_val)

# Evaluate individual models
print("Random Forest Accuracy:", accuracy_score(y_val, rf_preds))
print("XGBoost Accuracy:", accuracy_score(y_val, xgb_preds))
print("LightGBM Accuracy:", accuracy_score(y_val, lgbm_preds))

Random Forest Accuracy: 0.8324022346368715
XGBoost Accuracy: 0.8435754189944135
LightGBM Accuracy: 0.8379888268156425


In [ ]:
# Voting Classifier (Ensemble of all three models)

voting_model = VotingClassifier(
    estimators=[
        ('rf', rf_model)
        , ('xgb', xgb_model)
        , ('lgbm', lgbm_model)        
    ], voting='soft'
)
voting_model.fit(X_train, y_train)
voting_preds = voting_model.predict(X_val)

# Evaluate Voting Classifier
print("Voting Classifier Accuracy:", accuracy_score(y_val, voting_preds))

In [ ]:
# Combine models in a VotingClassifier (soft voting)
voting_clf = VotingClassifier(
    estimators=[
        ('rf', rf_clf)
        , ('xgb', xgb_clf)
        , ('lgb', lgb_clf)
    ]
    , voting='soft'
    , n_jobs=-1
)

In [ ]:
# NEED TO TWEEK THIS SIMILAR TO THE OTHER VOTING ENSEMBLE... 82% is too low for ensemble...


"""
# Initialize ensemble models (*Note: Since these are all tree methods, we did NOT standardize;)

rf_model = RandomForestClassifier(
    n_estimators=100, random_state=27
)
gb_model = GradientBoostingClassifier(
    n_estimators=100, random_state=27
)
xgb_model = XGBClassifier(
    n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=27
)


# Fit models individually

rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)


# Make predictions
rf_preds = rf_model.predict(X_test)
gb_preds = gb_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)

# Evaluate individual models
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print("Gradient Boosting Accuracy:", accuracy_score(y_test, gb_preds))
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_preds))

# Voting Classifier (Ensemble of all three models)

voting_model = VotingClassifier(
    estimators=[
        ('rf', rf_model)
        , ('gb', gb_model)
        , ('xgb', xgb_model)
    ], voting='soft'
)
voting_model.fit(X_train, y_train)
voting_preds = voting_model.predict(X_test)

# Evaluate Voting Classifier
print("Voting Classifier Accuracy:", accuracy_score(y_test, voting_preds))
"""

In [20]:
# Fit ensemble
voting_clf.fit(X_train, y_train)

,estimators,"[('rf', ...), ('xgb', ...), ...]"
,voting,'soft'
,weights,None
,n_jobs,-1
,flatten_transform,True
,verbose,False
,n_estimators,100
,criterion,'gini'
,max_depth,6
,min_samples_split,2
,min_samples_leaf,1


In [21]:

# Predict & evaluate on validation set
y_val_pred = voting_clf.predict(X_val)
val_acc = accuracy_score(y_val, y_val_pred)

val_acc

0.8212290502793296

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 199
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [ ]:
# voting_model = VotingClassifier(
#     estimators=[
#         ('rf', rf_model)
#         , ('gb', gb_model)
#         , ('xgb', xgb_model)
#     ], voting='soft'
# )
# voting_model.fit(X_train, y_train)
# voting_preds = voting_model.predict(X_test)

# # Evaluate Voting Classifier
# print("Voting Classifier Accuracy:", accuracy_score(y_test, voting_preds))